In [1]:
import pandas as pd
import tensorflow as tf
import nltk
import re
from nltk.corpus import stopwords

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\surya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
data = pd.read_csv('train.csv')

In [4]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
# Drop NAN Values present in the dataset.

data = data.dropna()

In [6]:
# Getting Independent Features present in the dataset.

X = data.drop('label', axis=1)
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [7]:
# Getting Dependent Features present in the dataset.

y = data['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


## Description of various layers used in this data.

## Embedding layers is used to perform Word Embedding so that similar words have similar embeddings.

## pad_sequences is used to ensure that all sentences have same length.

## Sequential layer is used to add Sequential model to the data layer.

## one_hot encoding is used to perform one hot encoding for the words.

## Dense layer is used to change the dimensions of the vectors used in each neural network layer.

## Dropout is used to reduce the overfitting of the neural network model.



In [11]:
# Initializing a vocabulary size.

voc_size = 7000

## Performing One-Hot Encoding for this dataset.

In [13]:
messages = X.copy()

In [16]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [17]:
# Reset index since we dropped NaN values in the previous step.

messages.reset_index(inplace=True)

## Data Pre-Processing.

## Porter Stemming is a process for removing the commoner morphological and inflexional endings from words in English.


In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0,len(messages)):
    clean_data = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    clean_data = clean_data.lower()
    clean_data = clean_data.split()
    
    # Performing Porter Stemming for words present in data excluding stopwords.
    clean_data = [ps.stem(word) for word in clean_data if word not in stopwords.words('english')]
    clean_data = ' '.join(clean_data)
    corpus.append(clean_data)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## Performing One-Hot Encoding for the words in corpus for our data.

In [24]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[514, 5376, 5012, 5192, 4324, 1195, 6062, 526, 1557, 4518],
 [5485, 4660, 2443, 5258, 411, 6318, 2352],
 [2065, 1328, 2520, 1967],
 [6761, 4940, 5407, 677, 157, 2424],
 [4949, 411, 4741, 1260, 3554, 4998, 411, 1011, 4071, 1158],
 [2437,
  5342,
  6875,
  2274,
  2477,
  1571,
  2444,
  2366,
  6990,
  5818,
  4256,
  2802,
  4987,
  2033,
  2352],
 [6009, 1220, 1795, 994, 931, 4452, 306, 4491, 6166, 5634, 148],
 [1150, 1617, 1006, 4055, 3115, 5159, 1571, 5834, 6166, 5634, 148],
 [2731, 1353, 97, 1481, 3951, 3566, 690, 898, 1571, 79],
 [2711, 4420, 3907, 5881, 1886, 941, 4455, 5416],
 [6165, 6268, 751, 993, 5727, 5126, 202, 5399, 442, 5080, 3892],
 [677, 5235, 4324, 3566, 1571, 3115],
 [2224, 1118, 5604, 6836, 5115, 4139, 3537, 1928, 5177],
 [6704, 1621, 4332, 6234, 2434, 371, 3741, 6166, 5634, 148],
 [6298, 1245, 5635, 1933, 17, 6166, 5634, 148],
 [5019, 842, 2025, 3983, 4467, 4282, 3815, 1188, 6419, 6126],
 [337, 5897, 4660],
 [3148, 6715, 3096, 1008, 1571, 560, 3649, 2352],
 [3691, 

## Embedding Representation using pad_sequences to make sure all sentences are embedded in same length.

In [25]:
sentence_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sentence_length)
embedded_docs

array([[   0,    0,    0, ...,  526, 1557, 4518],
       [   0,    0,    0, ...,  411, 6318, 2352],
       [   0,    0,    0, ..., 1328, 2520, 1967],
       ...,
       [   0,    0,    0, ..., 6166, 5634,  148],
       [   0,    0,    0, ..., 5796, 1049, 6608],
       [   0,    0,    0, ..., 5517, 4809,  841]])

In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  514,
       5376, 5012, 5192, 4324, 1195, 6062,  526, 1557, 4518])

In [28]:
## Creating a LSTM model.

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sentence_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            280000    
                                                                 
 dropout_1 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 336,501
Trainable params: 336,501
Non-trainable params: 0
_________________________________________________________________


In [30]:
import numpy as np

X_final = np.array(embedded_docs)
y_final = np.array(y)

In [31]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

## Performing train_test_split for our dataset.

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

## Fitting our model using training data and using testing data as validation data.

In [34]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=50)

Epoch 1/10
256/256 [==============================] - 12s 39ms/step - loss: 0.3024 - accuracy: 0.8580 - val_loss: 0.1894 - val_accuracy: 0.9171
Epoch 2/10
256/256 [==============================] - 11s 41ms/step - loss: 0.1372 - accuracy: 0.9484 - val_loss: 0.1965 - val_accuracy: 0.9154
Epoch 3/10
256/256 [==============================] - 11s 42ms/step - loss: 0.1008 - accuracy: 0.9627 - val_loss: 0.2131 - val_accuracy: 0.9160
Epoch 4/10
256/256 [==============================] - 10s 41ms/step - loss: 0.0700 - accuracy: 0.9759 - val_loss: 0.2475 - val_accuracy: 0.9178
Epoch 5/10
256/256 [==============================] - 9s 36ms/step - loss: 0.0491 - accuracy: 0.9844 - val_loss: 0.2736 - val_accuracy: 0.9085
Epoch 6/10
256/256 [==============================] - 9s 35ms/step - loss: 0.0338 - accuracy: 0.9895 - val_loss: 0.3141 - val_accuracy: 0.9092
Epoch 7/10
256/256 [==============================] - 9s 36ms/step - loss: 0.0296 - accuracy: 0.9901 - val_loss: 0.3968 - val_accuracy: 0.

## Prediction and Accuracy of our model using the embedded data.

In [39]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]])

In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[2863,  244],
       [ 286, 2093]], dtype=int64)

In [44]:
from sklearn.metrics import accuracy_score
print(f'Accuracy score:', accuracy_score(y_test, y_pred)*100)

Accuracy score: 90.3390448414145


## The Accuracy of our LSTM model is 90.33%.